# Hover Time

In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
# Load data
df = pd.read_csv("/Users/marlenerueschoff/Documents/Uni/UzK Master/Masterarbeit/Experiment/masterthesis_experiment-1/Data/mouse_events_rows (3).csv")

INPUT CSV (must contain):
  - participant_id : string
  - t_enter_ms     : number (ms since task start when entering area)
  - t_leave_ms     : number (ms since task start when leaving area)
  - dwell_ms       : number (optional; computed if missing as t_leave_ms - t_enter_ms)

OUTPUT:
  - hover_metrics_participant.csv
      columns: participant_id, total_ms, n_visits, mean_ms, median_ms, std_ms

In [4]:
# Validate required columns
required = {"participant_id", "t_enter_ms", "t_leave_ms", "duration_ms"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {', '.join(sorted(missing))}")

# Ensure numeric types
for col in ["t_enter_ms", "t_leave_ms"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")


#  Data Cleaning: keep valid, positive hover times and non-empty participant_id
df = df.loc[
    df["participant_id"].notna()
    & np.isfinite(df["duration_ms"])
    & (df["duration_ms"] > 0)
].copy()


# Aggregate per participant
metrics = (
    df.groupby("participant_id")["duration_ms"]
      .agg(total_hovertime_ms="sum",
           n_hovers="count",
           mean_hovertime_ms="mean",
           median_hovertime_ms="median",
           sd_hovertime_ms="std")      
      .reset_index()
)

# Replace NaN std with 0 for users with a single hover
metrics["sd_hovertime_ms"] = metrics["sd_hovertime_ms"].fillna(0.0)

# Calculate hovertime in seconds
metrics["total_hovertime_s"]  = metrics["total_hovertime_ms"]  / 1000.0
metrics["mean_hovertime_s"]   = metrics["mean_hovertime_ms"]   / 1000.0
metrics["median_hovertime_s"] = metrics["median_hovertime_ms"] / 1000.0
metrics["sd_hovertime_s"]    = metrics["sd_hovertime_ms"]    / 1000.0


# Print
display(metrics.head(10))


ValueError: Missing required columns: duration_ms, t_enter_ms, t_leave_ms